In [1]:
def SentimentAnalyzer(TickerName):
    from bs4 import BeautifulSoup
    from pprint import pprint
    import requests
    import pandas as pd
    from datetime import datetime

    url = "https://finviz.com/quote.ashx?t="+ str(TickerName)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    html_tables =[]
    for data in soup:
        data = soup.find(id='news-table')
        html_tables.append(data)

    # Hold the parsed news into a list
    parsed_news = []
    # Iterate through the news
    for news_table in html_tables:
        # Iterate through all tr tags in 'news_table'
        for x in news_table.findAll('tr'):
            # Read the text from the tr tag into text
            text = x.get_text() 
            # Split the text in the td tag into a list 
            date_scrape = x.td.text.split()
    #         print(date_scrape)
            # If the length of 'date_scrape' is 1, load 'time' as the only element
            # If not, load 'date' as the 1st element and 'time' as the second
            if len(date_scrape) == 1:
                time = date_scrape[0]
            else:
                date = date_scrape[0]
                time = date_scrape[1]

            parsed_news.append([date, time, x.a.text])


    # NLTK VADER for sentiment analysis
#     !pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    # New words and values
    new_words = {
        'crushes': 10,
        'beats': 5,
        'misses': -5,
        'trouble': -10,
        'falls': -100,
    }
    # Instantiate the sentiment intensity analyzer with the existing lexicon
    vader = SentimentIntensityAnalyzer()
    # Update the lexicon
    # nltk.download('stopwords')
    vader.lexicon.update(new_words)


    # Use these column names
    columns = ['date', 'time', 'headline']
    # Convert the list of lists into a DataFrame
    scored_news = pd.DataFrame(parsed_news,columns=columns)
    # Iterate through the headlines and get the polarity scores
    scores = [vader.polarity_scores(news[2]) for news in parsed_news]
    # Convert the list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)
    # Join the DataFrames
    scored_news = scored_news.join(scores_df)
    # Convert the date column from string to datetime
    scored_news['date'] = pd.to_datetime(scored_news.date).dt.date

    # Count the number of headlines in scored_news (store as integer)
    num_news_before = len(scored_news)
    # sum(scored_news.duplicated(subset=['headline']))
    # Drop duplicates based on ticker and headline
    scored_news_clean = scored_news.drop_duplicates(subset = ['headline'])
    # Count number of headlines after dropping duplicates
    num_news_after = len(scored_news_clean)
    # Print before and after numbers to get an idea of how we did 
#   print(f"Before we had {num_news_before} headlines, now we have {num_news_after}")
    print(scored_news_clean.head(20))


    # Set the index to ticker and date
    scored_news_date = scored_news_clean.set_index(['date'])
    # Group by mean of compound score of each date
    scored_news_date = scored_news_date.groupby('date').mean()
    print(scored_news_date['compound'])

    current_month = datetime.now().strftime('%B')
    TITLE = "Mean compound sentiment score for " + Ticker + " in " + current_month
    return scored_news_date['compound'].plot.bar(title=TITLE)

In [ ]:
Ticker = input("Ticker of the company?")
SentimentAnalyzer(Ticker)